In [2]:
import pandas as pd 
import yfinance as yf
import requests
from io import StringIO
import re

In [25]:
def load_nasdaq_tickers()-> pd.DataFrame:
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
    response = requests.get(url, timeout=30)
    lines = response.text.strip().split("\n") 
    lines = lines[:-1]  # Remove the last line which is a footer
    df = pd.read_csv(StringIO("\n".join(lines)),sep="|")
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    print(len(df), "before filtering")
    df = df[df["test_issue"] == "N"]
    df = df[df["etf"] == "N"]
    print(len(df), "after etf/test filtering")
    exclude_pattern = re.compile(r"unit|right|warrant|trust|fund|etf|etn|depositary", re.IGNORECASE)
    df = df[~df["security_name"].str.contains(exclude_pattern, na=False)]
    print(len(df), "after name filtering")
    df = df[df["security_name"].str.contains("common stock", case=False, na=False)]
    print(len(df), "after common stock filtering")
    print(len(df[df["market_category"] == "Q"]), "after market category filtering")
    df['Exchange_QID'] = "Q82059"
    df["YahooTicker"] = df["symbol"]
    return df

load_nasdaq_tickers()

5337 before filtering
4189 after etf/test filtering
3234 after name filtering
2331 after common stock filtering
1069 after market category filtering


,symbol,security_name,market_category,test_issue,financial_status,round_lot_size,etf,nextshares,Exchange_QID,YahooTicker
6,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100,N,N,Q82059,AAL
8,AAME,Atlantic American Corporation - Common Stock,G,N,N,100,N,N,Q82059,AAME
9,AAOI,"Applied Optoelectronics, Inc. - Common Stock",G,N,N,100,N,N,Q82059,AAOI
10,AAON,"AAON, Inc. - Common Stock",Q,N,N,100,N,N,Q82059,AAON
14,AAPL,Apple Inc. - Common Stock,Q,N,N,100,N,N,Q82059,AAPL
...,...,...,...,...,...,...,...,...,...,...
5322,ZSTK,ZeroStack Corp. - Common Stock,S,N,N,100,N,N,Q82059,ZSTK
5323,ZTEK,Zentek Ltd. - common stock,S,N,D,100,N,N,Q82059,ZTEK
5328,ZUMZ,Zumiez Inc. - Common Stock,Q,N,N,100,N,N,Q82059,ZUMZ
5330,ZVRA,"Zevra Therapeutics, Inc. - Common Stock",Q,N,N,100,N,N,Q82059,ZVRA


In [28]:
def load_nyse_tickers() -> pd.DataFrame:
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    lines = response.text.strip().split("\n") 
    lines = lines[:-1]  # Remove the last line which is a footer
    df = pd.read_csv(StringIO("\n".join(lines)),sep="|")
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    
    print(len(df), "before filtering")
    df = df[df["exchange"].isin(["N", "A", "P"])]
    print(len(df), "after filtering for exchange")
    df = df[df["etf"] == "N"]
    print(len(df), "after etf filtering")
    df = df[df["test_issue"] == "N"]
    print(len(df), "after test issue filtering")
    df = df[df["security_name"].str.contains("common stock", case=False, na=False)]
    print(len(df), "after common stock filtering")
    df["Exchange_QID"] = "Q13677"
    df['YahooTicker'] = df["cqs_symbol"]
    return df
load_nyse_tickers()

6986 before filtering
5772 after filtering for exchange
3152 after etf filtering
3133 after test issue filtering
1802 after common stock filtering


,act_symbol,security_name,exchange,cqs_symbol,etf,round_lot_size,test_issue,nasdaq_symbol,Exchange_QID,YahooTicker
0,A,"Agilent Technologies, Inc. Common Stock",N,A,N,100,N,A,Q13677,A
1,AA,Alcoa Corporation Common Stock,N,AA,N,100,N,AA,Q13677,AA
6,AAMI,Acadian Asset Management Inc. Common Stock,N,AAMI,N,100,N,AAMI,Q13677,AAMI
12,AAT,"American Assets Trust, Inc. Common Stock",N,AAT,N,100,N,AAT,Q13677,AAT
16,ABBV,AbbVie Inc. Common Stock,N,ABBV,N,100,N,ABBV,Q13677,ABBV
...,...,...,...,...,...,...,...,...,...,...
6961,ZIP,"ZipRecruiter, Inc. Class A Common Stock",N,ZIP,N,100,N,ZIP,Q13677,ZIP
6970,ZONE,CleanCore Solutions Inc. Class B Common Stock,A,ZONE,N,100,N,ZONE,Q13677,ZONE
6980,ZTS,Zoetis Inc. Class A Common Stock,N,ZTS,N,100,N,ZTS,Q13677,ZTS
6981,ZVIA,Zevia PBC Class A Common Stock,N,ZVIA,N,100,N,ZVIA,Q13677,ZVIA


In [ ]:
def load_euronext_tickers() -> pd.DataFrame:
    url = "https://live.euronext.com/pd_es/data/stocks/download"
    params = {
        "mics": "dm_all_stock",
        "issueType": "101", # Common Stocks
        "initialLetter": "",
        "display_datapoints": (
            "logo,name,isin,symbol,market,"
            "lastPrice,precentDayChange,lastTradeTime"
        ),
        "fe_type": "csv",
        "fe_decimal_separator": ".",
        "fe_date_format": "d/m/Y",
    }
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "text/csv",
    }

    # --- Download ---
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    csv_text = response.text

    # --- CSV korrekt einlesen ---
    df = pd.read_csv(
        StringIO(csv_text),
        sep=";", 
        encoding="utf-8-sig",
        low_memory=False,
    )
    df = df[3:] # Erste 3 Zeilen sind Metadaten

    # --- Erweitertes Mapping: Yahoo Suffix + Wikidata QID ---
    # Wir nutzen hier eine Struktur, die sowohl den Suffix als auch die WD-ID hält
    MARKET_CONFIG = {
        # Paris
        "Euronext Paris": {"suffix": ".PA", "qid": "Q2385849"},
        "Euronext Growth Paris": {"suffix": ".PA", "qid": "Q107188657"},
        "Euronext Access Paris": {"suffix": ".PA", "qid": "Q107190270"},
        # Amsterdam
        "Euronext Amsterdam": {"suffix": ".AS", "qid": "Q478720"},
        # Brussels
        "Euronext Brussels": {"suffix": ".BR", "qid": "Q1146518"}, # Gab zwei Ergebnisse
        "Euronext Growth Brussels": {"suffix": ".BR", "qid": "Q107189147"},
        "Euronext Access Brussels": {"suffix": ".BR", "qid": "Q107189150"},
        # Milan
        "Euronext Milan": {"suffix": ".MI", "qid": "Q936563"},
        "Euronext Growth Milan": {"suffix": ".MI", "qid": "Q23526757"},
        # Lisbon
        "Euronext Lisbon": {"suffix": ".LS", "qid": "Q2415561"},
        "Euronext Growth Lisbon": {"suffix": ".LS", "qid": "Q107189158"},
        "Euronext Access Lisbon": {"suffix": ".LS", "qid": "Q107188857"},
        # Dublin
        "Euronext Dublin": {"suffix": ".IR", "qid": "Q144458"}, #Gab zwei Ergebnisse
        "Euronext Growth Dublin": {"suffix": ".IR", "qid": "Q14Q107228013"},
        "Euronext Access Dublin": {"suffix": ".IR", "qid": "Q1435728"}, # Nicht gefunden
        # Oslo
        "Oslo Børs": {"suffix": ".OL", "qid": "Q909158"},
        "Euronext Growth Oslo": {"suffix": ".OL", "qid": "Q107190302"},
        "Euronext Expand Oslo": {"suffix": ".OL", "qid": "Q107188683"},
        # Multilistings
        "Euronext Brussels, Paris": {"suffix": ".BR", "qid": "Q2385845"},
        "Euronext Paris, Brussels": {"suffix": ".PA", "qid": "Q2385849"},
        "Euronext Brussels, Amsterdam": {"suffix": ".BR", "qid": "Q2385845"},
        "Euronext Amsterdam, Brussels": {"suffix": ".AS", "qid": "Q473938"},
        "Euronext Amsterdam, Paris": {"suffix": ".AS", "qid": "Q473938"},
        "Euronext Paris, Amsterdam": {"suffix": ".PA", "qid": "Q2385849"},
        "Euronext Paris, Amsterdam, Brussels": {"suffix": ".PA", "qid": "Q2385849"},
        "Euronext Amsterdam, Brussels, Paris": {"suffix": ".AS", "qid": "Q473938"},
        "Euronext Brussels, Amsterdam, Paris": {"suffix": ".BR", "qid": "Q2385845"},
        "Euronext Growth Brussels, Paris": {"suffix": ".BR", "qid": "Q2385845"},
        "Euronext Growth Paris, Brussels": {"suffix": ".PA", "qid": "Q2385849"},
    }
    Q107188657
    # Nur relevante Märkte behalten
    df = df[df['Market'].isin(MARKET_CONFIG.keys())]

    # --- Spalten generieren ---
    # Yahoo Ticker
    df["YahooTicker"] = df.apply(
        lambda row: f"{row['Symbol']}{MARKET_CONFIG[row['Market']]['suffix']}", 
        axis=1
    )
    
    # Wikidata QID (Börsenplatz)
    df["Exchange_QID"] = df["Market"].apply(
        lambda x: MARKET_CONFIG[x]["qid"]
    )

    return df
df_euronext = load_euronext_tickers()
df_euronext

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime,YahooTicker,Exchange_QID
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,131.10,133.50,130.60,133.20,13/02/2026 16:28,CET,209849,27799380.40,133.20,13/02/2026,2020.OL,Q909158
4,2CRSI,FR0013341781,AL2SI,Euronext Growth Paris,EUR,16.50,16.68,16.10,16.42,13/02/2026 17:35,CET,198833,3250417.13,16.42,13/02/2026,AL2SI.PA,Q107188657
11,4AIM SICAF,IT0005204729,AIM,Euronext Growth Milan,EUR,69.80,72.00,69.80,72.00,22/01/2026 15:34,CET,36,2522.00,72.00,13/02/2026,AIM.MI,Q23526757
12,4AIM SICAF COMP 2,IT0005440323,AIM2,Euronext Growth Milan,EUR,140.00,144.00,140.00,144.00,11/02/2026 17:25,CET,6,856.00,144.00,13/02/2026,AIM2.MI,Q23526757
13,5TH PLANET GAMES,DK0060945467,5PG,Euronext Expand Oslo,NOK,1.06,1.10,1.035,1.055,13/02/2026 15:57,CET,244527,264120.225,1.055,13/02/2026,5PG.OL,Q107188683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807,ZELLUNA,NO0013524942,ZLNA,Oslo Børs,NOK,15.954,15.954,15.564,15.564,13/02/2026 16:25,CET,10297,163405.72,15.564,13/02/2026,ZLNA.OL,Q909158
3808,ZENITH ENERGY,CA98936C8584,ZENA,Euronext Growth Oslo,NOK,0.453,0.456,0.43,0.453,13/02/2026 16:10,CET,2186939,972248.995,0.453,13/02/2026,ZENA.OL,Q107190302
3809,ZEST,IT0005013013,ZEST,Euronext Milan,EUR,0.1375,0.1375,0.1325,0.137,13/02/2026 17:35,CET,147950,19904.3665,0.137,13/02/2026,ZEST.MI,Q936563
3810,ZIGNAGO VETRO,IT0004171440,ZV,Euronext Milan,EUR,8.13,8.13,7.975,8.03,13/02/2026 17:35,CET,58971,473237.135,8.03,13/02/2026,ZV.MI,Q936563


In [41]:
df_euronext['Market'].value_counts()

Market
Euronext Paris                         314
Euronext Growth Paris                  251
Euronext Growth Milan                  209
Euronext Milan                         204
Oslo Børs                              198
Euronext Access Paris                  141
Euronext Amsterdam                     111
Euronext Brussels                       94
Euronext Growth Oslo                    81
Euronext Lisbon                         33
Euronext Dublin                         15
Euronext Access Lisbon                  14
Euronext Expand Oslo                    10
Euronext Access Brussels                 8
Euronext Growth Dublin                   7
Euronext Brussels, Paris                 6
Euronext Amsterdam, Brussels             5
Euronext Brussels, Amsterdam             5
Euronext Growth Brussels                 4
Euronext Paris, Brussels                 3
Euronext Amsterdam, Paris                2
Euronext Paris, Amsterdam, Brussels      2
Euronext Amsterdam, Brussels, Paris      1
Euro

In [47]:
nasdaq_t = load_nasdaq_tickers()[["symbol", "YahooTicker", "Exchange_QID"]]
print(len(nasdaq_t), "NASDAQ Tickers loaded")
nyse_t = load_nyse_tickers()[["cqs_symbol", "YahooTicker", "Exchange_QID"]]
euronext_t = load_euronext_tickers()[["Symbol", "YahooTicker", "Exchange_QID", "ISIN"]]
nasdaq_t.rename(columns={"symbol": "OriginalTicker"}, inplace=True)
nyse_t.rename(columns={"cqs_symbol": "OriginalTicker"}, inplace=True)
euronext_t.rename(columns={"Symbol": "OriginalTicker"}, inplace=True)
tickers =pd.concat([nasdaq_t, nyse_t, euronext_t], axis=0, ignore_index=True)

5337 before filtering
4189 after etf/test filtering
3234 after name filtering
2331 after common stock filtering
1069 after market category filtering
2331 NASDAQ Tickers loaded
6986 before filtering
5772 after filtering for exchange
3152 after etf filtering
3133 after test issue filtering
1802 after common stock filtering


In [45]:
tickers

,OriginalTicker,YahooTicker,Exchange_QID,ISIN
0,AAL,AAL,Q82059,NaN
1,AAME,AAME,Q82059,NaN
2,AAOI,AAOI,Q82059,NaN
3,AAON,AAON,Q82059,NaN
4,AAPL,AAPL,Q82059,NaN
...,...,...,...,...
5852,ZLNA,ZLNA.OL,Q909158,NO0013524942
5853,ZENA,ZENA.OL,Q107190302,CA98936C8584
5854,ZEST,ZEST.MI,Q936563,IT0005013013
5855,ZV,ZV.MI,Q936563,IT0004171440


In [46]:
tickers.to_csv("C:\\Diversification\\data\\tickers.csv", index=False)